In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Fri_Feb__8_19:08:17_PST_2019
Cuda compilation tools, release 10.1, V10.1.105


In [2]:
!nvidia-smi

Mon Jun 15 14:52:09 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.64       Driver Version: 430.64       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 00000000:03:00.0 Off |                  N/A |
| 26%   42C    P0    38W / 180W |      0MiB /  8119MiB |      2%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [1]:
import numpy as np
import time, os, sys
from urllib.parse import urlparse
import skimage
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300

import mxnet as mx
from cellpose import utils

use_GPU = utils.use_gpu()
print('GPU activated? %d'%use_GPU)

GPU activated? 1


'1.6.0'

In [ ]:
CSV_FOLDER = 'data/raw/CSV_patryk/'
TIFF_FOLDER = 'data/raw/RAW_TIFF/'
TEST_IMAGE = 'data/raw/RAW_TIFF/20072019_ND9_ND11_DIV0-1_Daam1_aTub_Phallo_10_ch_4.tif'
TEST_CSV = 'data/raw/CSV_patryk/20072019_ND9_ND11_DIV0-1_Daam1_aTub_Phallo_10_ch_4.csv'
BLOCK_ANNOTATION = 'data/raw/block_annotation2.csv'
BLOCK_SIZE = 40
TO_IGNORE = 'ND8_DIV0+2h_'

In [ ]:
use_gpu = utils.use_gpu()
if use_gpu:
    device = mx.gpu()
    print("use gpu")
else:
    device = mx.cpu()

In [4]:
# grayscale=0, R=1, G=2, B=3
# channels = [cytoplasm, nucleus]
Channels_map = {
        '20072019_ND9_ND11_DIV0-1_Daam1_aTub_Phallo': [3,2],
        'ND8_DIV0+2h': [2,1],
        'ND8_DIV0+4h_20x_noConfinment': [3,1]
    }

def getChannels(name):
    for key, channels in Channels_map.items():
        if name in key:
            return channels
    print('%s not found, return default [2,3]'%name)
    return [2,3]
        
getChannels('20072019_ND9_ND11_DIV0-1_Daam1')

def getPoint(csv,position):
    x = int(float(csv['x'][position]))
    y = int(float(csv['y'][position]))
    return x,y

In [5]:
def processAll(folder_csv,folder_tiff):
    X = []
    y = []
    all_csv,all_tiff = prepareList(folder_csv,folder_tiff,TO_IGNORE)
    all_csv= all_csv[:2]
    all_tiff= all_tiff[:2]
    annotations = pd.read_csv(BLOCK_ANNOTATION, header=None)
    channels = [getChannels(tmp) for tmp in all_tiff]
    all_mask,_,_,_ = getMasks(all_tiff,channels)
#     save(all_mask,all_tiff)
    for csv, tiff, mask in zip(all_csv, all_tiff,all_mask): 
        if getAnnotation(annotations, getName(tiff)) == -1:
            print("%s not exist in annotation !"%tiff)
        else:
            Xt, yt = processImage(csv,tiff,mask,annotations)
            X = X + Xt
            y = y + yt
    return X,y
 
# def checkExistInAnnotation(tif, annotation):
#     for t in annotation[0]:
def getName(path):
    elms = path.split('/')
    return elms[len(elms)-1].split('.')[0]    
    
def save(all_mask,all_tiff):
    for i in range(len(all_tiff)):
        name = os.path.basename(all_tiff[0]).replace('.tif','.jpg')
        cv2.imwrite('data/masks/%s'%name, all_mask[i])

def processImage(csv,tiff,mask,annotations):
    print('Processing %s'%os.path.basename(tiff))
    Xt = []
    yt = []
    data = pd.read_csv(csv)
    size = data.shape[0]
    vals = getVals(data,mask)
    image = plt.imread(tiff)
    base_name = os.path.basename(tiff).split('.')[0].replace("_ch_4", "")
    for i in range(size):
        x,y = getPoint(data,i)
        if i >= len(vals):
            print("%d more than max"%i)
            continue
        val = vals[i]
        block_name = '%s_%d'%(base_name,i)
        annotation = getAnnotation(annotations,block_name)
        if annotation == -1 :
            continue
        if val > 0 :
            activated_mask = activate(mask,val)
            cropped_mask = cropImage(activated_mask,x,y,BLOCK_SIZE)
            cropped_img = cropImage(image,x,y,BLOCK_SIZE)
            Xtmp = np.dstack((testing, cropped_mask))
            
#             activated_mask = np.stack((activated_mask,)*3, axis=-1)
#             masked_image = activateImage(image, activated_mask)
            height, width = image.shape
            if(checkPosition(height, width,y,x,BLOCK_SIZE)==False):
                continue
            cropped_image = cropImage(masked_image,x,y,BLOCK_SIZE)
            Xt.append(Xtmp)
            yt.append(annotation)
    print('%s created %d images'%(os.path.basename(tiff),len(Xt)))
    return Xt, yt

def getAnnotation(annotations, name):
    for file,val in zip(annotations[0],annotations[1]):
        if name in file:
            return val
    return -1
        
def checkPosition(height,width,y,x,size):
    if x<size :
        return False
    if y<size :
        return False
    if x+size>width :
        return False
    if y+size>height:
        return False
    return True

def cropImage(image,x,y,size):
    height, width = image.shape[0],image.shape[1]
    x1 = max([0,x-size])
    y1 = max([0,y-size])
    x2 = min([width,x+size])
    y2 = min([height,y+size])
#     base=np.zeros((size*2,size*2),dtype=np.uint8)
    return image[y1:y2, x1:x2]
    
def show(img,title):
    fig2 = plt.figure(figsize = (15,15))
    ax3 = fig2.add_subplot(111)
    ax3.imshow(img, interpolation='none')
    ax3.set_title(title)

def getPoint(data,i):
    return int(data['x'][i]),int(data['y'][i])

def activate(array,value):
    return (array == value) * 1

def activateImage(image,mask):
    return (mask == 1)*image

def getVals(data,mask):
    vals = []
    not_found = 0
    duplicate = 0
    for i in range(data.shape[0]):
        x,y = getPoint(data,i)
        val = mask[y][x]
        if val == 0 :
            not_found = not_found +1
            print("%d- %d Not found!"%(i,val))
        else:
            if val in vals:
                duplicate = duplicate +1
                print("%d- %d already exists!"%(i,val))
        vals.append(val)
    print("not found: %d"%not_found)
    print("duplicate: %d"%duplicate)
    return vals,not_found,duplicate

    
def prepareList(folder_csv,folder_tiff,to_ignore):
    all_csv = []
    all_tiff = []
    list_tiff = os.listdir(folder_tiff)
    
    for tiff in list_tiff:
        tiff_path = os.path.join(folder_tiff,tiff)
        csv = "%s.csv"%tiff.split('.')[0]
        csv_path = os.path.join(folder_csv,csv)
        if not os.path.exists(tiff_path):
            print('%s not exist'%tiff)
            continue
        if not os.path.exists(csv_path):
            print('%s not exist'%csv)
            continue
        if to_ignore in tiff :
            print('%s to ignore'%csv)
            continue
        all_csv.append(csv_path)
        all_tiff.append(tiff_path)
    return all_csv,all_tiff

def getMasks(list_tiff,channels):
    print('Detecting masks ..')
    model = models.Cellpose(device, model_type='cyto')
    imgs = [plt.imread(f) for f in list_tiff]
    nimg = len(imgs)
#     channels = [channel for i in range(len(list_tiff))]
    masks, flows, styles, diams = model.eval(imgs, rescale=None, channels=channels)
    print('masks detected.')
    return masks, flows, styles, diams



In [7]:
def getMainTiffFromBlock(name):
    fileName = name.split('.')[0]
    elms = fileName.split('_')
    nb = elms[len(elms)-1]
    elms = elms[:len(elms)-1]
    return '_'.join(elms),int(nb)

def loadAnnotation(path):
    annotation = pd.read_csv(path, header=None)
    annotation = annotation.sort_values(by=[0])
    annotation = annotation.reset_index(drop=True)
    return annotation

def processTif(path,name):
    img = plt.imread(path)
    channels = [getChannels(name)]
    mask,_,_,_ = getMasks([path],channels)
    return img, mask[0]

### Process
Go throw annotation and load images and generate 4d images : 3d + mask 80*80

In [9]:
Xt = []
Yt = []
skipped = 0
currentFile = ''
i = 0
annotation = loadAnnotation(BLOCK_ANNOTATION)
for ann,ytp in zip(annotation[0],annotation[1]):
     
    fileName,n = getMainTiffFromBlock(ann)
    if fileName != currentFile:
        i = 0
        if TO_IGNORE in fileName:
            continue
        if skipped > 0 :
            print("Total skipped %d/%d"%(skipped,i))
        print("Processing %s"%fileName)
        currentFile = fileName
        tifFile = TIFF_FOLDER + fileName+'.tif'
        img, mask = processTif(tifFile,currentFile)
        csvFile = CSV_FOLDER+fileName+'.csv'
        csv = pd.read_csv(csvFile, header=0)
        
    Ytmp = int(ytp)
    x,y = getPoint(csv,n)
#     print("x= %d , y= %d %s"%(x,y,mask.shape))
    if checkPosition(mask.shape[0],mask.shape[1],y,x,BLOCK_SIZE) == False:
        skipped = skipped + 1
        continue
        
    val = mask[y][x]
    activated_mask = activate(mask,val)
    cropped_mask = cropImage(activated_mask,x,y,BLOCK_SIZE)
    cropped_image = cropImage(img,x,y,BLOCK_SIZE)
    cropped_image = cropped_image / 255.
    Xtmp = np.dstack((cropped_image, cropped_mask))
    
    Xt.append(Xtmp)
    Yt.append(Ytmp)
    i = i+1
    
#     print(csv)
#     print(x)
#     break

Processing 20072019_ND9_ND11_DIV0-1_Daam1_aTub_Phallo_10_ch_4
20072019_ND9_ND11_DIV0-1_Daam1_aTub_Phallo_10_ch_4 not found, return default [2,3]
Detecting masks ..
masks detected.
Total skipped 10/0
Processing 20072019_ND9_ND11_DIV0-1_Daam1_aTub_Phallo_11_ch_4
20072019_ND9_ND11_DIV0-1_Daam1_aTub_Phallo_11_ch_4 not found, return default [2,3]
Detecting masks ..
masks detected.
Total skipped 11/0
Processing 20072019_ND9_ND11_DIV0-1_Daam1_aTub_Phallo_12_ch_4
20072019_ND9_ND11_DIV0-1_Daam1_aTub_Phallo_12_ch_4 not found, return default [2,3]
Detecting masks ..
masks detected.
Total skipped 18/0
Processing 20072019_ND9_ND11_DIV0-1_Daam1_aTub_Phallo_13_ch_4
20072019_ND9_ND11_DIV0-1_Daam1_aTub_Phallo_13_ch_4 not found, return default [2,3]
Detecting masks ..
masks detected.
Total skipped 30/0
Processing 20072019_ND9_ND11_DIV0-1_Daam1_aTub_Phallo_14_ch_4
20072019_ND9_ND11_DIV0-1_Daam1_aTub_Phallo_14_ch_4 not found, return default [2,3]
Detecting masks ..
masks detected.
Total skipped 34/0
Proce

In [10]:
import pickle
Xt = np.array(Xt).reshape(-1,80,80,4)
Yt = np.array(Yt).reshape(-1,1)



pickle_out = open("data/X2_4d_masked.pickle","wb")
pickle.dump(Xt,pickle_out)
pickle_out.close()

pickle_out = open("data/y2_4d_masked.pickle","wb")
pickle.dump(Yt,pickle_out)
pickle_out.close()


In [47]:
Xt[0].shape

(80, 80, 4)

###  Archive

In [ ]:
# for idx in range(nimg):
#     img = transforms.reshape(imgs[idx], channels[idx])
#     img = plot.rgb_image(img)
#     maski = masks[idx]
#     flowi = flows[idx][0]

#     fig = plt.figure(figsize=(12,3))
#     # can save images (set save_dir=None if not)
#     plot.show_segmentation(fig, img, maski, flowi)
#     plt.tight_layout()
#     plt.show()

In [ ]:
# show(imgs[0],'Original')
# show(masks[0],'Mask')
# show(flows[0][0],'Flow')

In [ ]:
# activated_mask = activate(masks[0],110)
# print(activated_mask.max())
# activated_mask = np.stack((activated_mask,)*3, axis=-1)
# print(activated_mask.shape)
# show(activated_mask,'Mask')

In [ ]:
# number_segments = masks[0].max()
# # number_segments
# for i in range(1,20):
#     x = activate(masks[0],i)
#     stacked_img = np.stack((x,)*3, axis=-1)
#     dst = activateImage(image, stacked_img)
#     show(dst,str(i))

In [50]:
# X,y = processAll(CSV_FOLDER,TIFF_FOLDER)
# print(X.shape)
# print(y.shape)

In [51]:
# import cv2
# image = imgs[0]
# dst = activateImage(image, stacked_img)
# show(dst,'Mask')

In [53]:
# annotation = pd.read_csv(BLOCK_ANNOTATION, header=None)

In [54]:
# print(point)
# for i in range(csv.shape[0]):
#     x,y = getPoint(csv,i)
#     val = mask[y][x]
#     activated_mask = activate(mask,val)
# #     show(activated_mask,"cropped")
#     cropped_mask = cropImage(activated_mask,x,y,BLOCK_SIZE)
#     show(cropped_mask,"cropped")
#     cropped_image = cropImage(img,x,y,BLOCK_SIZE)
#     show(cropped_image,"cropped")
#     np.dstack((testing, cropped_mask)).shape